In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

**keys**: PEC scattering, single and double layer potential, MoM, unknown Neumann trace (surface current density)

Maxwell PEC Direct method
=============================

We consider a perfect conductor $\Omega \subset \mathbb R^3$ and an electromagnetic signal that induces a scattered electric field which propagates into $\Omega^c$. The scattered electric field $\boldsymbol E$ solves the following Dirichlet boundary value problem: 

$$ \left\{ \begin{array}{rcl l} \mathbf{curl} \, \mathbf{curl}\, \boldsymbol E - \kappa^2 \, \boldsymbol E &=& \boldsymbol 0, \quad &\textnormal{in } \Omega^c \subset \mathbb R^3\,,\\ \gamma_R \,\boldsymbol E &=& \boldsymbol m, \quad & \textnormal{on }\Gamma \\ \left\| \mathbf{curl} \, \boldsymbol E( x) - i\,\omega\,\epsilon \, \boldsymbol E( x)\right\| &=& \mathcal O\left( \displaystyle \frac{1}{\| x\|^2}\right), &\textnormal{for} \; \|x\| \to \infty\,.\end{array} \right. $$ 


A possible representation for $\boldsymbol E$ reads

$$ \boldsymbol E(x) = \underbrace{ \kappa \, \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}s_y  +  \frac{1}{\kappa} \, \nabla \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{SL}(\boldsymbol j)}}+ \underbrace{ \nabla \times  \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol n \times \boldsymbol{m}(y)\, \mathrm{d}s_y }_{\displaystyle \mathrm{DL}(\boldsymbol n\times\boldsymbol m) } \,.$$

**NGSolve Solution**  
Define the geometry of the perfect conductor $\Omega$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(3)

Next, we create finite element spaces for $\boldsymbol H^{-\frac12}(\mathrm{curl}_\Gamma, \Gamma)$ (test functions) and $\boldsymbol H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma)$ (trial functions): 

In [ ]:
order = 3
fesHCurl = HCurl(mesh, order=order, complex=True)
fesHDiv = HDivSurface(mesh, order=order, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) trial for Dirichlet data ( (nxE)xn )
uHDiv,vHDiv = fesHDiv.TnT() # H(div_Gamma) trial space for Neumann data ( nx curlE ) and test space for BIE

print ("ndof HCurl = ", fesHCurl.ndof)
print ("ndof HDiv = ", fesHDiv.ndof)

**Boundary Integral Equation**

We carefully apply the tangential trace $\gamma_R$ on $\boldsymbol E$ and obtain a boundary integral equation for the unknown trace $\boldsymbol j$. The boundary integral equation is solved by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \begin{array}{l c c c c c c} \forall \, \boldsymbol v\in H^{-\frac12}(\mathrm{div}_\Gamma, \Gamma): &  & \displaystyle \int\limits_\Gamma \mathrm{SL}(\boldsymbol j) \cdot \boldsymbol v \, \mathrm d s &=&  \displaystyle \int\limits_\Gamma \boldsymbol m \cdot  \boldsymbol v \, \mathrm d s &+&  \displaystyle \int\limits_\Gamma \mathrm{DL}(\boldsymbol n \times \boldsymbol{m}) \cdot \boldsymbol v \, \mathrm d s \\[2ex] & \Rightarrow & \mathrm{V}\,\mathrm j &=& \displaystyle \frac12\,\mathrm{M} \, \mathrm m &-&  \mathrm K\, \mathrm m \,. \end{array} $$ 

In the enineering community, the approximation scheme is also known as **method of moments** (MoM). 

Define the incoming plane wave and compute the given boundary data $\boldsymbol m = -\gamma_R \boldsymbol E_{\mathrm{inc}}$ : 

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E_inc = CF((1,0,0))*exp( -1j * kappa * z )

n = specialcf.normal(3)
m = GridFunction(fesHCurl) 
m.Set( -E_inc, definedon=mesh.Boundaries(".*"), dual=True) # Hcurl

The discretisation of the above variational formulation leads to a system of linear equations, ie
$$ \mathrm{V} \, \mathrm j = \left( \frac12 \,\mathrm{M} - \mathrm{K} \right)\, \mathrm{m}\,,$$
where  
* $\mathrm V$ is the Maxwell single layer operator. $V$ is a regular, symmetric matrix.
* $\mathrm{M}$ is the mass matrix.
* $\mathrm K$ is the Maxwell double layer operator. $K$ is quadratic:

The assembly of the potential operators is declarative and close to their actual implementations. All contributions to the Maxwell layer potentials are derivates of the Helmholtz layer potentials and thus, we use the Helmholtz operators as building blocks (explicite formulas are given at the end of the notebook).

In [ ]:
intorder = 2 * order + 6
with TaskManager(): 
    M = BilinearForm( uHCurl.Trace()*vHDiv.Trace()*ds(bonus_intorder=3)).Assemble() # <Hcurl, Hdiv> 

In [ ]:
with TaskManager(): 
    V1 = HelmholtzSL( uHDiv.Trace()*ds , kappa) * vHDiv.Trace() * ds
    V2 = HelmholtzSL( div(uHDiv.Trace()) * ds, kappa) * div(vHDiv.Trace()) * ds
    V = kappa * V1.mat - 1/kappa * V2.mat
    K = HelmholtzDL(uHCurl.Operator("rotated_trace") * ds, kappa) * vHDiv.Trace() * ds

In [ ]:
# solve direct formulation for j, i.e., Neumann data of scattered electric field E
j = GridFunction(fesHDiv) 
pre = BilinearForm( uHDiv.Trace() * vHDiv.Trace() *ds).Assemble().mat.Inverse(freedofs=fesHDiv.FreeDofs()) 
with TaskManager(): 
    rhs = ( (0.5 * M.mat - K.mat ) * m.vec).Evaluate() 
    GMRes(A = V, pre=pre, b = rhs, x=j.vec, tol=1e-8, maxsteps=5000, printrates=True)

In [ ]:
Draw(j[1].real, mesh, draw_vol=False, order=3);

**TODO Evaluate the Solution**

To evaluate the solution inside $\Omega$ we have to evaluate the representation formula, i.e., 

$$ \boldsymbol E(x) = \kappa \, \underbrace{\int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol j(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{L}_1(\boldsymbol j) } }  +  \frac{1}{\kappa} \nabla \underbrace{ \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol j(y)\, \mathrm{d}s_y }_{\displaystyle{\mathrm{L}_2(\boldsymbol j)} } + \nabla \times \underbrace{\int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol n \times \boldsymbol{m}(y)\, \mathrm{d}s_y }_{\displaystyle{ \mathrm{L}_3 (\boldsymbol{m}) } } \,.$$

The kernel function that appears in the Maxwell layer potentials is the same as for Helmholtz. Indead, it turns out that the Maxwell layer potentials are derivates of Helmholtz layer potentials. We make this relation clear by using the Helmholtz layer potentials and layer potential operators as building blocks for Maxwell operators. Note that there are formulations such as the low frequency stabilization that even require an explicit split of the summands of the Maxwell single layer potential (References below).

$$ \begin{array}{rcl} \mathrm{L}_1(\boldsymbol u ) &=& \verb-HelmholtzSL(u * ds, kappa)- \\
\mathrm{L}_2(\boldsymbol u) &=& \verb-HelmholtzSL(div(u)*ds, kappa)- \\
\mathrm{L}_3(\boldsymbol u) &=& \verb-HelmholtzSL(u.Operator("rotated_trace") *ds, kappa)-\,. \end{array}$$

**Details on Layer Potential Operators**

For a trial function $\boldsymbol u_j \in H^{-\frac12}(\mathrm{div}_\Gamma,\Gamma)$ and a test function $\boldsymbol v_i \in H^{-\frac12}(\mathrm{div}_\Gamma,\Gamma)$ the Maxwell single layer potential operator entry $V_{ij}$ can formally written as

$$ V_{ij} = \kappa \, \int\limits_\Gamma \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \boldsymbol u_j(y) \cdot \boldsymbol v_i(x) \, \mathrm{d}s_x \mathrm{d} s_x - \frac{1}{\kappa} \int\limits_\Gamma \int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi}\, \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} } \, \mathrm{div}_\Gamma \boldsymbol u_j(y)\,  \mathrm{div}_\Gamma \boldsymbol v_i(x)\, \mathrm{d}\sigma_y\,\mathrm d s_x\,. $$ 

The two contributions are derivates of the Helmholtz single layer potential operator and may be implemented as such:   

```
V = kappa * (HelmholtzSL(u*ds, kappa)*v*ds - 1/kappa * HelmholtzSL(div(u)*ds, kappa)*div(v)*ds
```

For a trial function $\boldsymbol u_j \in H^{-\frac12}(\mathrm{curl}_\Gamma,\Gamma)$ and a test function $\boldsymbol v_i \in H^{-\frac12}(\mathrm{div}_\Gamma,\Gamma)$ the Maxwell double layer potential operator entry $K_{ij}$ can formally written as

$$ K_{ij} = \int\limits_\Gamma \displaystyle\int\limits_\Gamma \displaystyle{ \frac{1}{4\,\pi} \, \nabla_x \frac{e^{i\,\kappa\,\|x-y\|}}{\| x-y\|} }  \times  ( \boldsymbol n \times \boldsymbol{u}_j(y) ) \, \mathrm{d} s_y \cdot \boldsymbol v_i(x) \mathrm d s_x\,.$$

Giving transparency to its relation to the Helmholtz double layer potential operator, you may assemble it as follows:


```
K = HelmholtzDL( u.Operator("rotated_trace") * ds, kappa) * v * ds
```



**References (theoretical and numerical results):**

- [High order Boundary Element Methods](https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/26312/1/thesis_weggler_final_6.1.12.pdf)
- [Stabilized boundary element methods for low-frequency electromagnetic scattering](https://onlinelibrary.wiley.com/doi/10.1002/mma.1597)
